In [1]:
import requests
import json
import re
import secrets
import os

import pandas as pd
import numpy as np

from openpyxl import load_workbook

In [2]:
PROJECT_SETTINGS_PATH = 'settings.xlsx'

In [3]:
FOLDERS_PATH = '{}'

## Получение списка проектов из Settings.xlsx

### Одна карта

In [4]:
def get_projects_with_one_map():
    return pd.read_excel(PROJECT_SETTINGS_PATH)

### Мультикарта

In [5]:
def get_projects_with_many_maps():
    return pd.read_excel(PROJECT_SETTINGS_PATH, sheet_name = 'many_maps')

### Экспорт новых данных на определенный лист в Settings.xlsx

In [6]:
def export_data_to_specified_sheet(data, export_sheet_name):
    with pd.ExcelWriter(PROJECT_SETTINGS_PATH, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
        data.to_excel(writer, export_sheet_name, index=False)

## Генерация ключей и запись в таблицу

In [7]:
def create_private_key_and_export_settings_xlsx(project_settings, export_sheet_name):
    for row in project_settings:
        row['key'] = secrets.token_hex(64)
        
    updated_project_settings = pd.DataFrame.from_dict(project_settings)
    
    export_data_to_specified_sheet(updated_project_settings, export_sheet_name)
    
    

## Создание папок

In [8]:
def create_folders(project_settings):
    for row in project_settings:
        os.makedirs(FOLDERS_PATH.format(row['project']),exist_ok=True)

## Создание txt с ключами

In [9]:
def create_private_key_txt(project_name, project_txt_name, key):
    with open(os.path.join(FOLDERS_PATH.format(project_name), f'{project_txt_name}_private_key_indexnow.txt'), 'w') as f:
            f.write(key)

## Подготовка пустой CSV базы

In [10]:
def create_empty_csv_base(project_folder, project_csv_name):
    data = {
    'url':np.nan,
    'status':np.nan,
    }
    
    base = pd.DataFrame(data, index=[0])
    base.to_csv(f'{project_folder}/{project_csv_name}_url_base_indexnow.csv',index=False)

## Главная функция - генерация ключей, создание папок, txt и csv файлов

In [11]:
def set_up_projects_first_time_main():
    #one_map
    project_settings = get_projects_with_one_map().to_dict('record')
    create_private_key_and_export_settings_xlsx(project_settings,'one_map')
    create_folders(project_settings)
    
    project_settings = get_projects_with_one_map().to_dict('record')
    for row in project_settings:
        project, key = row['project'], row['key']
        create_private_key_txt(project, project, key)
        create_empty_csv_base(project, project)
    
    #many_maps
    project_settings = get_projects_with_many_maps().to_dict('record')
    create_private_key_and_export_settings_xlsx(project_settings,'many_maps')
    create_folders(project_settings)
    
    project_settings = get_projects_with_many_maps().to_dict('record')
    for row in project_settings:
        project, key = row['project'], row['key']
        create_private_key_txt(project, project, key)
        create_empty_csv_base(project, project)


## Запуск

In [12]:
set_up_projects_first_time_main()

/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  import sys
/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning

## Дополнение базы поддоменами из Яндекс Вебмастера

### Токен и URL

In [13]:
TOKEN = 'Ваш токен'

USERID_URL = 'https://api.webmaster.yandex.net/v4/user'
GET_SITE_LIST = 'https://api.webmaster.yandex.net/v4/user/{}/hosts'

### Авторизация

In [14]:
def get_auth_headers() -> dict:
    return {'Authorization': f'OAuth {TOKEN}'}

### Получение User ID

In [15]:
def get_user_id() -> str:
    r = requests.get(USERID_URL, headers=get_auth_headers())
    user_id = json.loads(r.text)['user_id']
    return user_id

### Получение списка хостов по каждому проекту из Settings
#### Пример написания хоста https:site.ru:443

In [16]:
def request_yandex_webmaster(user_id, project):
    request = requests.get(GET_SITE_LIST.format(user_id), headers=get_auth_headers())
    site_list = json.loads(request.text)
    
    project_subdomains_hosts = []
    for number in range(len(site_list['hosts'])):
        regex = re.search(f'.*{project}.*', site_list['hosts'][number]['host_id'])
        
        if regex:
            project_subdomains_hosts.append(regex.group())
    
    return project_subdomains_hosts

### Извлечение поддоменов из хостов

In [17]:
def extract_subdomains_from_hosts(project_subdomains_hosts):
    project_subdomains_list = []
    for domain in project_subdomains_hosts:
        project_subdomains_list.append(domain.split(':')[1])
        
    return project_subdomains_list

### Добавление поддоменов в Settings

In [18]:
def add_subdomains_to_settings(project_settings, project_subdomains_list, export_sheet_name):
    list_of_dicts = []
    
    for domain in project_subdomains_list:
        new_dict = {}
        new_dict['project'] = domain
        new_dict['domain_type'] = 'subdomain'
        new_dict['main_domain_name'] = domain.split('.', 1)[1]
        list_of_dicts.append(new_dict)
        
    for element in list_of_dicts:
        project_settings.append(element)
    
    updated_project_settings = pd.DataFrame.from_dict(project_settings)
    
    updated_project_settings.drop_duplicates(subset=['project'], inplace=True)
    
    export_data_to_specified_sheet(updated_project_settings, export_sheet_name)

In [19]:
def get_subdomains_from_webmaster_update_settings(user_id, project_settings, project_settings_slice, export_sheet_name):
    for row in project_settings_slice:
        project = row['project']
        
        project_subdomains_hosts = request_yandex_webmaster(user_id, project)
        project_subdomains_list = extract_subdomains_from_hosts(project_subdomains_hosts)
        
        add_subdomains_to_settings(project_settings, project_subdomains_list, export_sheet_name)
        

### Главная функция получение поддоменов из Яндекс Вебмастера и добавление в Settings

In [20]:
def update_settings_with_subdomains_main():
    user_id = get_user_id()
    
    #one_map
    projects_one_map = get_projects_with_one_map()
    projects_one_map_slice = projects_one_map[projects_one_map['domain_type'] == 'main'].to_dict('record')
    
    projects_one_map = projects_one_map.to_dict('record')
    get_subdomains_from_webmaster_update_settings(user_id, projects_one_map, projects_one_map_slice, 'one_map')
    
    #many_maps
    projects_many_maps = get_projects_with_many_maps()
    projects_many_maps_slice = projects_many_maps[projects_many_maps['domain_type'] == 'main'].to_dict('record')
    
    projects_many_maps = projects_many_maps.to_dict('record')
    get_subdomains_from_webmaster_update_settings(user_id, projects_many_maps, projects_many_maps_slice, 'many_maps')

## Запуск

In [21]:
update_settings_with_subdomains_main()

/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  
/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  
/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  del sys.path[0]
/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipyk

## Создание ключей для поддоменов

### Создание ключей и запись в таблицу

In [22]:
def replace_empty_key_value_in_settings(project_settings, sheet_name):
    project_settings = project_settings.to_dict('record')

    for row in project_settings:
        if type(row['key']) != str:
            row['key'] = secrets.token_hex(64)

    projects_subdomain_with_keys = pd.DataFrame.from_dict(project_settings)
    export_data_to_specified_sheet(projects_subdomain_with_keys, sheet_name)

### Создание txt и добавление в папку по названию проекта

In [23]:
def create_txt(project_settings, projects_subdomains_without_key_list):
    
    projects_for_file_creating = project_settings[project_settings['project'].isin(projects_subdomains_without_key_list)]

    projects_for_file_creating = projects_for_file_creating.to_dict('record')

    for row in projects_for_file_creating:
        subdomain, key, main_domain = row['project'], row['key'], row['main_domain_name']
        create_private_key_txt(main_domain, subdomain, key)

### Главная функция - создание ключей и добавление в таблицу, создание txt и добавление в папку

In [24]:
def create_data_for_subdomains_main():
    #one_map
    projects_one_map = get_projects_with_one_map()
    ##Список поддоменов без ключа
    projects_subdomains_without_key_list = projects_one_map[projects_one_map['key'].isna()]['project'].to_list()
    
    replace_empty_key_value_in_settings(projects_one_map, 'one_map')
    
    projects_one_map = get_projects_with_one_map()
    create_txt(projects_one_map, projects_subdomains_without_key_list)
    
    #many_maps
    projects_many_maps = get_projects_with_many_maps()
    #Список поддоменов без ключа
    projects_subdomains_without_key_list = projects_many_maps[projects_many_maps['key'].isna()]['project'].to_list()
    
    replace_empty_key_value_in_settings(projects_many_maps, 'many_maps')
    
    projects_many_maps = get_projects_with_many_maps()
    create_txt(projects_many_maps, projects_subdomains_without_key_list)

## Запуск

In [25]:
create_data_for_subdomains_main()

/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  
/Users/andreytryastsin/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.
  """
